# Second Hands-On Session
--- 
Tracking many particles - Twiss parameters

---

## 🐍 Python corner

We'll be using:
- `numpy` as `np`
- `matplotlib.pyplot` as `plt`
- functions `D`

In [1]:
from tracking_library import *

numpy is installed, version: 2.3.1
scipy is installed, version: 1.15.3
matplotlib is installed, version: 3.10.0
-> Setup is OK! Have fun!


## 📝 Exercise 2.1 (guided): Generating an ensemble of particles <a id="exercise-2.1-guided"></a>

Define an ensemble of 100 particles with arbitrary first order (e.g. $<x_0> = 0.2$ mm, $<x'_0> = 1$ mrad) and second order (e.g. $\sigma_{x_0} = 1.5$ mm and $\sigma_{x'_0} = 0.5$ mrad) momenta.
Verify that the angular divergence of the beam is the one set: 

- What do you observe?
- What happens if you increase or decrease the number of particles?

> 👀 **HINT**: Remember that you can create a Normal distributed 2xN 2D array of numbers as `np.random.randn(2, N)`. One can then "stretch" and "offset" (and eventually "rotate", but this is not requested here) this distribution to obtain any desired 2D normal distribution in phase space.

In [3]:
# simple solution
N_particles = 100
beam = np.array([np.random.randn(N_particles), np.random.randn(N_particles)])
# or equivalently:
beam = np.random.randn(2, N_particles)
x0 = 0.2
xp0 = 1
sigx = 1.5
sigxp = 0.5
beam[0, :] = sigx * beam[0, :] + x0
beam[1, :] = sigxp * beam[1, :] + xp0
print(f"Beam mean  x [mm]: {np.mean(beam[0,:])}")
print(f"Beam mean  x' [mrad]: {np.mean(beam[1,:])}")
print(f"Beam rms size [mm]: {np.std(beam[0,:])}")
print(f"Beam rms divergence [mrad]: {np.std(beam[1,:])}")

Beam mean  x [mm]: 0.24374210497644114
Beam mean  x' [mrad]: 1.0192541598536438
Beam rms size [mm]: 1.6008511207095986
Beam rms divergence [mrad]: 0.4767701056857321


❓**QUESTION: Why didn't we find back the input parameters?**

## 📝 Exercise 2.2: Tracking an ensemble of particles <a id="exercise-2.2"></a>

1. Transport the beam distribution of [Exercise 2.1](#exercise-2.1-guided) in a drift of length 1 m. Visualise the initial and final distribution. **What do you observe?**

   > 👀 **HINT 1**: the output of `transportParticles` contains the coordinates of all particles at all locations. To obtain all particle coordinates at the end of the beamline, you can simply do:
   >
   > ```
   > tracked = transportParticles(initial_coordinates, beamline)
   > final_coordinates = tracked['coords'][-1] #where "-1" refers to the last element of the beamline
   > ```

   > 👀 **HINT 2**: One can use the `seaborn` package to nicely see a 2D distribution, e.g.:
   > ```
   > import seaborn as sns
   > sns.jointplot(x=np.random.randn(1000), y=np.random.randn(1000), kind="hex", color="b")
   > ```

2. Test of linearity. Scale the input vector by 17 times the month of your birthday (e.g. 85 if you are born in May) and verify that the output vector from the matrix multiplication has changed by the same factor.
    
    > 👀 **HINT**: Be careful with machine precision!


In [ ]:
# code here your solution...

## 📝 Exercise 2.3: Beam size & divergence evolution along a beamline <a id="exercise-2.3"></a>

Build a beamline made of several drift and quadrupoles as desired (e.g. `D(0.5)+Q(1)+D(2)+Q(-1)+D(1)+Q(2)+....`).
Build a beam made of several particles (e.g. 1000) again with arbitrary first order (e.g. $<x_0> = 0.2$ mm, $<x'_0> = 1$ mrad) and second order (e.g. $\sigma_{x_0} = 1.5$ mm and $\sigma_{x'_0} = 0.5$ mrad) momenta, as done in previous [Exercise 2.2](#exercise-2.2).

Compute and plot the beam $size$ ($\sigma_x$) and $divergence$ ($\sigma_{x'}$) along the beam line.

> 👀 **HINT 1 (Python)**: Remember that in the output of our `transportParticles` function the key `'x'` contains a 2D array with N rows (the index of the position along the beam line) and M columns (the index of a given particle). 
You can compute the standard deviation of **each raw** of a NxM 2D array as `np.std(N_times_M_array,1)`. 

> 👀 **HINT 2 (Python)**: After having plotted $x$ trajectory on a matplotlib plot, one can create a **second vertical axis** that shares the same horizontal axis with `plt.twinx()`. This can be convenient to see, for example, both $\sigma_x$ and $\sigma_{x'}$ on the same plot.


In [ ]:
# code here your solution...

## ⚛️ Physics focus: Sigma matrices

One can easily demonstrate (see [Wolfgan's lecture](https://indico.cern.ch/event/1356988/contributions/5713241/)) that the same matrix ($M$) used for tracking the coordinates ($(x_i, x'_i)$) of each single particle ($i$) from an initial point ($X_0$) to a final point ($X_s$) in a beamline:

\begin{equation}
X_s =  M\, X_0
\end{equation}

can also be used to track the **average trajectory** ($\langle X \rangle$) as well as the **covariance or sigma matrix** of the given particle coordinates distribution:

\begin{equation}
\langle X_s \rangle = 
\left[
\begin{array}{c}
\langle x_i \rangle\\
\langle x'_i \rangle
\end{array}
\right]_s 
= M\, \langle X_0 \rangle
\end{equation}

\begin{equation}
\Sigma_s = \left[
\begin{array}{c}
\sigma_{xx}\quad \sigma_{xx'}\\
\sigma_{x'x}\quad \sigma_{x'x'}
\end{array}
\right]_s
= M\, \Sigma_0\, M^T\, .
\end{equation}

We can therefore track the **average trajectory** and **covariance** of a beam simply starting from its initial average coordinates and covariance matrix in phase space.

The "tracking" of an initial **covariance** matrix along a given beamline is provided by the function `transportSigmas()` function from our toolbox:

In [4]:
from tracking_library import transportSigmas

# let's see if there is some help information:
help(transportSigmas)

Help on function transportSigmas in module tracking_library:

transportSigmas(sigma_0, beamline)
    Transport a sigma matrix through a beamline.

    Parameters
    ----------
    sigma_0 : ndarray
        Initial 2x2 sigma matrix.
    beamline : list of dict
        Sequence of beamline elements.

    Returns
    -------
    result : dict
        Dictionary with keys:
            'sigma11', 'sigma12', 'sigma21', 'sigma22' : [N] sigma matrix elements,
            's'                                        : [N] longitudinal positions,
            'sigmas'                                   : [N x 2 x 2] full sigma matrices.

    Disclaimer
    -------
    If beamline is made of 5 elements, the output will have 5+1 "elements" as it will also return include the initial particle coordinates.



## 📝 Exercise 2.4: Tracking the average beam position

Show that the average position of a beam made of 100 particles along a beam line (e.g. the beamline you defined in [Exercise 2.3](#exercise-2.3)) is the same as the the trajectory of single particle particle that starts in the center of the initial particle distribution.

In [ ]:
# code here your solution...

## 📝 Excercise 2.5: Tracking the beam size

For the same system as before, compare the rms beam size computed from tracking all particles and from using the sigma matrix approach.

> 🔹 **NOTE**: Is this valid for any number of initially tracked particles? How does the result change if one uses the **input** covariance matrix used to generate the particle distribution rather then the **actual** covariance matrix of the generated distribution?

> 👀 **HINT 1**: Remember that the element $\sigma_{xx}$ of the covariance matrix is linked to the rms beam size ($\sigma_x$) as $\sigma_x = \sqrt{\sigma_{xx}}$.

> 👀 **HINT 2**: The covariance matrix of a 2xN array can be computed using `numpy` as `np.cov(2_times_N_array, bias=True)`.

In [ ]:
# code here your solution...

## ⚛️ Physics focus: Introduction of Twiss values and emittance

The sigma/covariance matrix of any particle distribution can also be written as:

\begin{equation}
\Sigma = 
    \left[
    \begin{array}{cc}
    \sigma_{xx}  & \sigma_{xx'}\\
    \sigma_{x'x} & \sigma_{x'x'}
    \end{array}
    \right] =
    \epsilon
    \left[ 
    \begin{array}{cc}
        \beta   & -\alpha\\
        -\alpha & \gamma
    \end{array}
    \right] 
\end{equation}

where $\beta$, $\alpha$, $\gamma$ and $\epsilon$ are parameters such that $\epsilon = \sqrt{\det(\Sigma)}$ and $\beta \gamma - \alpha^2 = 1$. 

**This seems to be an arbitrary choice!** but it will acquire more special meaning later.

In the meantime, recall once more that:

\begin{equation}
\Sigma_{s} = M\, \Sigma_{0}\, M^T\, .
\end{equation}

where $M$ is a real **symplectic** transformation, and its determinant is $\det(M) = +1$, therefore:

\begin{equation}
\epsilon_s = \det(\Sigma_s) = \det( M\, \Sigma_0\, M^T ) = \det(M) \det(\Sigma_0) \det(M^T) = \det(\Sigma_0) = \epsilon_0
\end{equation}

**Two important considerations:** 
- the quantity $\epsilon$ (the beam **statistical emittance**) is **preserved** along a beamline!!!
- the *Twiss* parmeters $\beta, \gamma, \alpha$ define the **normalised** shape/orientation of the beam distribution in phase-space!

## 📝 Exercise 2.6: Equivalent matrix symplecticity
Verify that the equivalent transport matrix of any beamline, e.g. the one you used in [Exercise 2.3](#exercise-2.3), has determinant equal to 1 (within machine precision).

> 👀 **HINT (Python)**: you can use `np.linalg.det(matrix)` to compute the determinant of a matrix `matrix`

In [ ]:
# code here your solution...

## 📝 Exercise 2.7: Evolution of Twiss parameters in a beamline

Consider again a beamline, e.g. the one you used in [Exercise 2.3](#exercise-2.3), and create a valid sigma matrix with:
- $\beta$ = 3 [m]
- $\gamma$ = 0.5 [1/m]
- $\epsilon$ = 5 [$\mu$ m]

Then, propagate the $\sigma$ matrix through the beam line and verify that the emittance $\epsilon$ of the sigma matrix after every element is indeed constant and equal to its initial value.

**Optional:** compute and plot the **beta** (function) all along the beamline, i.e. $\sigma_{11}/\epsilon$

> 👀 **HINT (Python)**: in the output of our `transportSigmas()` function we keep all sigma matrixes. The determinant of all matrices can be computed in one go as `np.linalg.det(transported_sigmas['sigmas'])`.

In [ ]:
# code here your solution...

## Well done !!

Now you understand how to transport an ensamble of particles or its **covariance** matrix along a beamline... But how to design a "nice" beamline?

=> **Continue your learning with the following [notebook](./03_Periodic_Systems.ipynb)**...
